# Module 

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import tensorflow
sys.path.append("../")
# from src.data import load_data

# Config

In [2]:
debug = True

if debug:
    dataset = "../data_model/aclImdb/debug"
    BATCH_SIZE = 1
else:
    dataset = "../data_model/aclImdb/"
    BATCH_SIZE = 128
pretrained_path = '../data_model/uncased_L-12_H-768_A-12'
SEQ_LEN = 128
EPOCHS = 5
LR = 1e-4
# os.environ['TF_KERAS'] = '1'

In [3]:
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [4]:
# @title Convert Data to Array  
import codecs
from keras_bert import load_trained_model_from_checkpoint

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

        import os
import numpy as np
from tqdm import tqdm
from keras_bert import Tokenizer

tokenizer = Tokenizer(token_dict)


def load_data(path):
    global tokenizer
    indices, sentiments = [], []
    for folder, sentiment in (('neg', 0), ('pos', 1)):
        folder = os.path.join(path, folder)
        for name in tqdm(os.listdir(folder)):
            with open(os.path.join(folder, name), 'r') as reader:
                  text = reader.read()
            ids, segments = tokenizer.encode(text, max_len=SEQ_LEN)
            indices.append(ids)
            sentiments.append(sentiment)
    items = list(zip(indices, sentiments))
    np.random.shuffle(items)
    indices, sentiments = zip(*items)
    indices = np.array(indices)
    mod = indices.shape[0] % BATCH_SIZE
    if mod > 0:
        indices, sentiments = indices[:-mod], sentiments[:-mod]
    return [indices, np.zeros_like(indices)], np.array(sentiments)
  
train_path = os.path.join(dataset, 'train')
test_path = os.path.join(dataset, 'test')


train_x, train_y = load_data(train_path)
test_x, test_y = load_data(test_path)

Using TensorFlow backend.
100%|██████████| 2/2 [00:00<00:00, 171.83it/s]


# Modle 

## load model 

In [5]:
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,
)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 128)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 128)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 128, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 128, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [8]:
# @title Build Custom Model
from tensorflow.python import keras
from keras_radam import RAdam

inputs = model.inputs[:2]
dense = model.get_layer('NSP-Dense').output
outputs = keras.layers.Dense(units=2, activation='softmax')(dense)
# outputs = tensorflow.keras.layers.Dense(units=2, activation='softmax')(dense)


model = keras.models.Model(inputs, outputs)
model.compile(
    RAdam(lr=LR),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

AttributeError: 'Dense' object has no attribute 'outbound_nodes'

In [0]:
# @title Initialize Variables
import tensorflow as tf
import tensorflow.keras.backend as K

sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

In [0]:
# @title Fit

model.fit(
    train_x,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
)

Epoch 1/5
195/195 [==============================] - 84s 432ms/step - loss: 0.7142 - sparse_categorical_accuracy: 0.5161
Epoch 2/5
195/195 [==============================] - 38s 196ms/step - loss: 0.6928 - sparse_categorical_accuracy: 0.5467
Epoch 3/5
195/195 [==============================] - 38s 195ms/step - loss: 0.6061 - sparse_categorical_accuracy: 0.6643
Epoch 4/5
195/195 [==============================] - 38s 195ms/step - loss: 0.4952 - sparse_categorical_accuracy: 0.7586
Epoch 5/5
195/195 [==============================] - 38s 195ms/step - loss: 0.4192 - sparse_categorical_accuracy: 0.8067


In [0]:
# @title Predict

predicts = model.predict(test_x, verbose=True).argmax(axis=-1)

In [0]:
# @title Accuracy

print(np.sum(test_y == predicts) / test_y.shape[0])